In [1]:
import enum
import glob
import os
from hashlib import new
from pathlib import Path
import time
from itertools import product

import functools

import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from snorkel.labeling.model import LabelModel as LMsnorkel
from snorkel.labeling.model import MajorityLabelVoter

from sklearn.model_selection import train_test_split
import itertools
import ast
import time 

from tqdm import tqdm

In [2]:
# Initialize seed
seed_i = 0

In [3]:
import joblib
import json
import collections

In [4]:
from sklearn.exceptions import UndefinedMetricWarning

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [5]:
def list2Nested(l, nested_length):
    return [l[i:i+nested_length] for i in range(0, len(l), nested_length)]

In [6]:
labelModel_mapper_LF = {1:1, -1:0, 0:-1}

In [7]:
import LMutils

train_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/train_ebm_labels_tui_pio3.tsv'
training_data = pd.read_csv(train_file, sep='\t', header=0)

ebm_test_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/test_ebm_labels_tui_pio3.tsv'
test_ebm_data = pd.read_csv(ebm_test_file, sep='\t', header=0)
test_ebm_data.rename( columns={'Unnamed: 0':'series'}, inplace=True )

physio_test_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/test_physio_labels_tui_pio3.tsv'
test_physio_data = pd.read_csv(physio_test_file, sep='\t', header=0)
test_physio_data.rename( columns={'Unnamed: 0':'series'}, inplace=True )

ebm_test_corrected_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/test_ebm_correctedlabels_tui_pio3.tsv'
test_ebm_corrected_data = pd.read_csv(ebm_test_corrected_file, sep='\t', header=0)
test_ebm_corrected_data.rename( columns={'Unnamed: 0':'series'}, inplace=True )

In [8]:
def flatten_df(df):

    df_series = [ index for index, value in df.tokens.items() for word in ast.literal_eval(value) ]
    df_tokens = [ word for index, value in df.tokens.items() for word in ast.literal_eval(value) ]
    df_pos = [ word for index, value in df.pos.items() for word in ast.literal_eval(value) ]
    df_offsets = [ word for index, value in df.offsets.items() for word in ast.literal_eval(value) ]


    df_p = [ int(lab) for index, value in df.p.items() for lab in ast.literal_eval(value) ]
    df_p_fine = [ int(lab) for index, value in df.p_f.items() for lab in ast.literal_eval(value) ]
    df_i = [ int(lab) for index, value in df.i.items() for lab in ast.literal_eval(value) ]
    df_i_fine = [ int(lab) for index, value in df.i_f.items() for lab in ast.literal_eval(value) ]
    df_o = [ int(lab) for index, value in df.o.items() for lab in ast.literal_eval(value) ]
    df_o_fine = [ int(lab) for index, value in df.o_f.items() for lab in ast.literal_eval(value) ]
    df_s = [ int(lab) for index, value in df.s.items() for lab in ast.literal_eval(value) ]
    df_s_fine = [ int(lab) for index, value in df.s_f.items() for lab in ast.literal_eval(value) ]
    
    df_flattened = pd.DataFrame({ 'series': df_series,
                        'tokens' : df_tokens,
                        'offsets': df_offsets,
                        'pos': df_pos,
                        'p' : df_p,
                        'i' : df_i,
                        'o' : df_o,
                        's' : df_s,
                        'p_f' : df_p_fine,
                        'i_f' : df_i_fine,
                        'o_f' : df_o_fine,
                        's_f' : df_s_fine})
    
    return df_flattened

In [9]:
# Flatten the dataframes (currently only the training dataframe and test ebm dataframe with corrected labels can be flattened)
data_df = flatten_df(training_data)
test_ebm_data = flatten_df(test_ebm_data)
test_ebm_corr_df = flatten_df(test_ebm_corrected_data)

In [10]:
series = [
    data_df.series.to_numpy() ,
    test_ebm_data.series.to_numpy() ,
    test_physio_data.series.to_numpy(),   
    test_ebm_corr_df.series.to_numpy()
]


sents = [
    data_df.tokens.to_numpy() ,
    test_ebm_data.tokens.to_numpy() ,
    test_physio_data.tokens.to_numpy(),   
    test_ebm_corr_df.tokens.to_numpy()    
]


part_of_speech = [
    data_df.pos.to_numpy() ,
    test_ebm_data.pos.to_numpy() ,
    test_physio_data.pos.to_numpy(),   
    test_ebm_corr_df.pos.to_numpy()     
]


offsets = [
    data_df.offsets.to_numpy() ,
    test_ebm_data.offsets.to_numpy() ,
    test_physio_data.offsets.to_numpy(),   
    test_ebm_corr_df.offsets.to_numpy() 
]


Y_p = [
    data_df.p.to_numpy() , # 0 -7
    data_df.p_f.to_numpy() , # 1 -6
    test_ebm_data.p.to_numpy() , # 2 -5
    test_ebm_data.p_f.to_numpy() , # 3 -4
    test_physio_data.p.to_numpy(),  # 4 -3
    test_ebm_corr_df.p.to_numpy(),   # 5 -2
    test_ebm_corr_df.p_f.to_numpy() # 6 -1
]


Y_i = [
    data_df.i.to_numpy() , # 0 -7
    data_df.i_f.to_numpy() , # 1 -6
    test_ebm_data.i.to_numpy() , # 2 -5
    test_ebm_data.i_f.to_numpy() , # 3 -4
    test_physio_data.i.to_numpy(),  # 4 -3
    test_ebm_corr_df.i.to_numpy(),   # 5 -2
    test_ebm_corr_df.i_f.to_numpy() # 6 -1
]


Y_o = [
    data_df.o.to_numpy() , # 0 -7
    data_df.o_f.to_numpy() , # 1 -6
    test_ebm_data.o.to_numpy() , # 2 -5
    test_ebm_data.o_f.to_numpy() , # 3 -4
    test_physio_data.o.to_numpy(),  # 4 -3
    test_ebm_corr_df.o.to_numpy(),   # 5 -2
    test_ebm_corr_df.o_f.to_numpy() # 6 -1
]

Y_s = [
    data_df.s.to_numpy() , # 0 -7
    data_df.s_f.to_numpy() , # 1 -6
    test_ebm_data.s.to_numpy() , # 2 -5
    test_ebm_data.s_f.to_numpy() , # 3 -4
    test_physio_data.s.to_numpy(),  # 4 -3
    test_ebm_corr_df.s.to_numpy(),   # 5 -2
    test_ebm_corr_df.s_f.to_numpy() # 6 -1
]

In [11]:
# Write data for error analysis

error_analysis_ebm_p = pd.DataFrame({'tokens' : test_ebm_data.tokens,
                                'participant' : test_ebm_data.p,
                                'participant_fine' : test_ebm_data.p_f }, 
                                columns=['tokens','participant', 'participant_fine'])

#error_analysis_ebm_p.to_csv (r'/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/error_analysis/test_ebmgold_p', index = None, header=True) 

In [12]:
# Write data for error analysis

error_analysis_ebmcorr_p = pd.DataFrame({'tokens' : test_ebm_corr_df.tokens,
                                'participant' : test_ebm_corr_df.p,
                                'participant_fine' : test_ebm_corr_df.p_f }, 
                                columns=['tokens','participant', 'participant_fine'])

#error_analysis_ebmcorr_p.to_csv (r'/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/error_analysis/test_ebmgoldcorr_p', index = None, header=True) 

In [13]:
def df_to_list(data_column):
    return [ word for index, value in data_column.items() for word in ast.literal_eval(value) ]

In [14]:
def df_to_array(data_column):
    return np.array( [ word for index, value in data_column.items() for word in ast.literal_eval(value) ] )

In [15]:
def dict_to_array(label_column):
    return np.array( [ labelModel_mapper_LF[int(lab)] for index, value in label_column.items() for k, lab in ast.literal_eval(value).items() ] )

In [16]:
def get_lfs(indir):
    
    pathlist = Path(indir).glob('**/*.tsv')

    tokens = ''

    lfs = dict()
    lfs_lm = dict()

    for counter, file in enumerate(pathlist):
        #print(file)
       
        if '/O/' in str(file) and '_negs' not in str(file):
            #print(file)

            k = str( file ).split('/v4/')[-1].replace('.tsv', '').replace('/', '_')
            mypath = Path(file)
            if mypath.stat().st_size != 0:
                data = pd.read_csv(file, sep='\t', header=0)

                data_tokens = data.tokens
                if len(tokens) < 5:
                    tokens = df_to_array(data_tokens)

                data_labels = data.labels
                labels = dict_to_array(data_labels)
                if len(labels) != len(tokens):
                    print(k, len(labels) , len(tokens) )
                lfs[k] = labels


    print( 'Total number of tokens in validation set: ', len(tokens) )
    print( 'Total number of LFs in the dictionary', len(lfs) )
    
    return lfs

Total number of tokens in validation set:  1303169 <br>
Total number of LFs in the dictionary 277

In [17]:
indir = '/mnt/nas2/results/Results/systematicReview/distant_pico/training_ebm_candidate_generation/v4'
train_ebm_lfs = get_lfs(indir)

Total number of tokens in validation set:  1303169
Total number of LFs in the dictionary 289


In [18]:
indir_test_ebm_corr = '/mnt/nas2/results/Results/systematicReview/distant_pico/test_ebm_anjani_candidate_generation/v4'
test_ebm_corr_lfs = get_lfs(indir_test_ebm_corr)

Total number of tokens in validation set:  52582
Total number of LFs in the dictionary 289


In [19]:
indir_test_ebm = '/mnt/nas2/results/Results/systematicReview/distant_pico/test_ebm_candidate_generation/v4'
test_ebm_lfs = get_lfs(indir_test_ebm)

Total number of tokens in validation set:  51784
Total number of LFs in the dictionary 289


In [20]:
indir_test_physio = '/mnt/nas2/results/Results/systematicReview/distant_pico/test_physio_candidate_generation/v4'
test_physio_lfs = get_lfs(indir_test_physio)

Total number of tokens in validation set:  52504
Total number of LFs in the dictionary 289


In [21]:
# Remove the annotations where there are no positive labels
print('Dropping no positive label LFs')

def drop_nopositive(lfs_d):
    
    dropped_no_positives = dict()

    for k, v in lfs_d.items():

        if len(set(v)) < 3:
            if 1 in list(set(v)):
                dropped_no_positives[k] = v
        else:
            dropped_no_positives[k] = v
            
    return dropped_no_positives

#lfs_dropped = drop_nopositive(lfs)
#print('Number of LFs: ', len(lfs_dropped))
#lfs_ebm_corr_dropped = drop_nopositive(test_ebm_corr_lfs)
#print('Number of LFs: ', len(lfs_ebm_corr_dropped))

Dropping no positive label LFs


In [22]:
def lf_levels(umls_d:dict, pattern:str, picos:str):

    umls_level = dict()

    for key, value in umls_d.items():   # iter on both keys and values
        search_pattern = pattern + picos
        if key.startswith(search_pattern):
            k = str(key).split('_')[-1]
            umls_level[ k ] = value

    return umls_level


# Level 1: UMLS
umls_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testphysio = [
    lf_levels(test_physio_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

umls_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testphysio = [
    lf_levels(test_physio_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

umls_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]
def lf_levels(umls_d:dict, pattern:str, picos:str):

    umls_level = dict()

    for key, value in umls_d.items():   # iter on both keys and values
        search_pattern = pattern + picos
        if key.startswith(search_pattern):
            k = str(key).split('_')[-1]
            umls_level[ k ] = value

    return umls_level


# Level 1: UMLS
umls_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

umls_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

umls_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


# Level 2: non UMLS
nonumls_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------


nonumls_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

nonumls_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

# Level 3: DS
ds_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]


# ------------------------------------------------------------------------

ds_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

# ------------------------------------------------------------------------

ds_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


# Level 4: dictionary, rules, heuristics
heur_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]


heur_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_s = [
    lf_levels(train_ebm_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_s_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]


heur_s_testebm = [
    lf_levels(test_ebm_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


dict_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_s = [
    lf_levels(train_ebm_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testebm = [
    lf_levels(test_ebm_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]
umls_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o_testphysio = [
    lf_levels(test_physio_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


# Level 2: non UMLS
nonumls_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testphysio = [
    lf_levels(test_physio_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------


nonumls_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testphysio = [
    lf_levels(test_physio_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

nonumls_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testphysio = [
    lf_levels(test_physio_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

# Level 3: DS
ds_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testphysio = [
    lf_levels(test_physio_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]


# ------------------------------------------------------------------------

ds_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testphysio = [
    lf_levels(test_physio_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

# ------------------------------------------------------------------------

ds_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testphysio = [
    lf_levels(test_physio_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


# Level 4: dictionary, rules, heuristics
heur_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testphysio = [
    lf_levels(test_physio_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testphysio = [
    lf_levels(test_physio_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]


heur_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_o_testphysio = [
    lf_levels(test_physio_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_s = [
    lf_levels(train_ebm_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_s_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]


heur_s_testebm = [
    lf_levels(test_ebm_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_s_testphysio = [
    lf_levels(test_physio_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


dict_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testphysio = [
    lf_levels(test_physio_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testphysio = [
    lf_levels(test_physio_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testphysio = [
    lf_levels(test_physio_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_s = [
    lf_levels(train_ebm_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testebm = [
    lf_levels(test_ebm_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testphysio = [
    lf_levels(test_physio_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

In [23]:
# Participant data

train_candidates = [umls_p[1], nonumls_p[1], ds_p[1], dict_p[1], heur_p[0]]
test_ebm_corr_candidates = [umls_p_testcorrected[1], nonumls_p_testcorrected[1], ds_p_testcorrected[1], dict_p_testcorrected[1], heur_p_testcorrected[0]]
test_ebm_candidates = [umls_p_testebm[1], nonumls_p_testebm[1], ds_p_testebm[1], dict_p_testebm[1], heur_p_testebm[0]]
test_physio_candidates = [umls_p_testphysio[1], nonumls_p_testphysio[1], ds_p_testphysio[1], dict_p_testphysio[1], heur_p_testphysio[0]]

In [24]:
# Intervention data

train_i_candidates = [umls_i[1], nonumls_i[1], ds_i[1], dict_i[1], heur_i[0]]
test_i_ebm_corr_candidates = [umls_i_testcorrected[1], nonumls_i_testcorrected[1], ds_i_testcorrected[1], dict_i_testcorrected[1], heur_i_testcorrected[0]]
test_i_physio_candidates = [umls_i_testphysio[1], nonumls_i_testphysio[1], ds_i_testphysio[1], dict_i_testphysio[1], heur_i_testphysio[0]]

In [25]:
# Outcome data

train_o_candidates = [umls_o[1], nonumls_o[1], ds_o[1], dict_o[1], heur_o[0]]
test_o_ebm_corr_candidates = [umls_o_testcorrected[1], nonumls_o_testcorrected[1], ds_o_testcorrected[1], dict_o_testcorrected[1], heur_o_testcorrected[0]]
test_o_ebm_candidates = [umls_o_testebm[1], nonumls_o_testebm[1], ds_o_testebm[1], dict_o_testebm[1], heur_o_testebm[0]]
test_o_physio_candidates = [umls_o_testphysio[1], nonumls_o_testphysio[1], ds_o_testphysio[1], dict_o_testphysio[1], heur_o_testphysio[0]]

In [26]:
# Study Type data

train_s_candidates = [dict_s[1], heur_s[0]]
test_s_ebm_corr_candidates = [dict_s_testcorrected[1], heur_s_testcorrected[0]]
test_s_ebm_candidates = [dict_s_testebm[1], heur_s_testebm[0]]
test_s_physio_candidates = [dict_s_testphysio[1], heur_s_testphysio[0]]

In [27]:
def getLen(lf):
    for l in lf:
        for k,v in l.items():
            print(len(v))
            
#getLen(umls_i) 
#getLen(nonumls_i) 

In [28]:
# Fetch UMLS ranks

sum_lf_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_p_summary_tuipio3_train.csv'
sum_lf_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_i_summary_tuipio3_train.csv'
sum_lf_o = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_o_summary_tuipio3_train.csv'


def fetchRank(sum_lf_d, pattern, picos: str, drop_nopos: bool):
    
    drop_nopos_keys = []
    for k,v in train_ebm_lfs.items():
        query = '_'+picos+'_'
        if query in str(k):
            key = str(k).split('_lf_')[-1]
            drop_nopos_keys.append(key)
    
    ranked_umls_coverage = dict()    
    umls_coverage_ = dict()
    
    data=pd.read_csv(sum_lf_d, sep='\t')
    
    for index, row in data.iterrows():
        if row[0].startswith(pattern):
            umls_coverage_[row[0]] = row[3]
    
    umls_coverage_sorted = sorted(umls_coverage_.items(), key=lambda x: x[1], reverse=True)
    
    for i in umls_coverage_sorted:
        k = str(i[0]).split('_')[-1]
        if drop_nopos == False:
            ranked_umls_coverage[k] = i[1]
        else:
            if k in drop_nopos_keys:
                ranked_umls_coverage[k] = i[1]

    return ranked_umls_coverage


# fuzzy UMLS
ranksorted_p_umls_fuzzy = fetchRank(sum_lf_p, 'UMLS_fuzzy_', picos='P', drop_nopos = False)
ranksorted_i_umls_fuzzy = fetchRank(sum_lf_i, 'UMLS_fuzzy_', picos='I', drop_nopos = False)
ranksorted_o_umls_fuzzy = fetchRank(sum_lf_o, 'UMLS_fuzzy_', picos='O', drop_nopos = False)


# direct UMLS
ranksorted_p_umls_direct = fetchRank(sum_lf_p, 'UMLS_direct_', picos='P', drop_nopos = False)
ranksorted_i_umls_direct = fetchRank(sum_lf_i, 'UMLS_direct_', picos='I', drop_nopos = False)
ranksorted_o_umls_direct = fetchRank(sum_lf_o, 'UMLS_direct_', picos='O', drop_nopos = False)

proper_coverage_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/coverage_results/participant_UMLS_v3_coverage.json'

with open(proper_coverage_p, 'r') as rf:
    data = json.load(rf)
 
    for k, v in data.items():
        if k in ranksorted_p_umls_fuzzy:
            ranksorted_p_umls_fuzzy[k] = data[k]
            
ranksorted_coverage_p_umls_fuzzy = sorted(ranksorted_p_umls_fuzzy.items(), key=lambda x: x[1], reverse=True)
ranksorted_coverage_p_umls_fuzzy = dict(ranksorted_coverage_p_umls_fuzzy)


proper_coverage_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/coverage_results/intervention_UMLS_v3_coverage.json'

with open(proper_coverage_i, 'r') as rf:
    data = json.load(rf)
 
    for k, v in data.items():
        if k in ranksorted_i_umls_fuzzy:
            ranksorted_i_umls_fuzzy[k] = data[k]
            
ranksorted_coverage_i_umls_fuzzy = sorted(ranksorted_i_umls_fuzzy.items(), key=lambda x: x[1], reverse=True)
ranksorted_coverage_i_umls_fuzzy = dict(ranksorted_coverage_i_umls_fuzzy)

In [29]:
# Partition LF's

def partitionLFs(umls_d):
    
    keys = list(umls_d.keys())

    partitioned_lfs = [ ]
    
    for i in range( 0, len(keys) ):

        if i == 0 or i == len(keys):
            if i == 0:
                partitioned_lfs.append( [keys] )
            if i ==len(keys):
                temp3 = list2Nested(keys, 1)
                partitioned_lfs.append( temp3 )
        else:
            temp1, temp2 = keys[:i] , keys[i:]
            temp3 = list2Nested( keys[:i], 1)
            temp3.append( keys[i:] )
            partitioned_lfs.append( temp3 )
    
    return partitioned_lfs


partitioned_p_umls_fuzzy = partitionLFs(ranksorted_coverage_p_umls_fuzzy)
partitioned_i_umls_fuzzy = partitionLFs(ranksorted_coverage_i_umls_fuzzy)
partitioned_o_umls_fuzzy = partitionLFs(ranksorted_o_umls_fuzzy)

partitioned_p_umls_direct = partitionLFs(ranksorted_p_umls_direct)
partitioned_i_umls_direct = partitionLFs(ranksorted_i_umls_direct)
partitioned_o_umls_direct = partitionLFs(ranksorted_o_umls_direct)

In [30]:
#exp_level = ['UMLS', 'UMLS_Ontology', 'UMLS_Ontology_Rules']
#exp_level = ['UMLS_Ontology_Rules', 'UMLS_Ontology', 'UMLS']
exp_level = ['UMLS_Ontology_Rules', 'UMLS_Ontology', 'UMLS']

In [31]:
param_grid = {
    'lr': [0.001, 0.0001],
    'l2': [0.001, 0.0001],
    'n_epochs': [50, 100, 200, 600, 700, 1000, 2000],
    'prec_init': [0.6, 0.7, 0.8, 0.9],
    'optimizer': ["adamax", "adam", "sgd"],
    'lr_scheduler': ['constant'],
}

In [32]:
def sample_param_grid(param_grid, seed):
    """ Sample parameter grid
    :param param_grid:
    :param seed:
    :return:
    """
    rstate = np.random.get_state()
    np.random.seed(seed)
    params = list(product(*[param_grid[name] for name in param_grid]))
    np.random.shuffle(params)
    np.random.set_state(rstate)
    return params

In [33]:
def compare(s, t):
    return sorted(s) == sorted(t)

def getLFs(partition:list, umls_d:dict, seed_len:int):

    all_lfs_combined = []
    
    for lf in partition: # for each lf in a partition
        
        combine_here = [0] * seed_len

        for sab in lf:
            new_a = list(umls_d[sab])
            old_a = combine_here
            temp_a = []
            
            for o_a, n_a in zip(old_a, new_a):
                               
                if compare([o_a, n_a] ,[-1, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                    
                elif compare([o_a, n_a] ,[0, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                    
                elif compare([o_a, n_a] ,[-1, 0]) == True:
                    replace_a = min( o_a, n_a )
                    temp_a.append( replace_a )
                else:
                    temp_a.append( o_a )

            combine_here = temp_a

        all_lfs_combined.append( combine_here )

    return all_lfs_combined

In [34]:
import time

def compare(s, t):
    return sorted(s) == sorted(t)

def getLFs_modified(partitions:list, umls_d:dict, seed_len:int):
    
    start = time.time()

    all_lfs_combined = []
    
    for part in partitions: # for each lf in a partition
        
        merged_lfs = list( map(umls_d.get, part) )
        merged_lfs = np.stack( merged_lfs )
        merged_lfs = pd.DataFrame( merged_lfs )
        merged_lfs = merged_lfs.transpose()
        merged_lfs = merged_lfs.to_dict('records')

        emitted = []
        
        #for index, row in merged_lfs.iterrows(): # NEVER USE ITERROWS
        for row in merged_lfs:
            
            emit = ''
            to_compare = list( set( list(row.values())  ) )
            if len( to_compare ) == 3:
                emit = max( to_compare )
            else:
                if 1 in to_compare:
                    emit = max( to_compare )
                else:
                    emit = min( to_compare )
                    
            emitted.append( emit )
            
        all_lfs_combined.append( emitted )
    
    end = time.time()
    
    #print('End time: ', end - start)

    return all_lfs_combined

In [35]:
# Dump the partitions to a fine
picos = 'O'
candgen_version = 'v4'

def write_parition( data_dict, setname, k ):
    write_dir = f'/mnt/nas2/results/Results/systematicReview/distant_pico/partitions/{candgen_version}/{picos}/{setname}/'
    filename = str(k) + '.json'
    write_file = write_dir + filename
    with open(write_file , 'w+') as wf:
         json.dump(data_dict, wf)

def part2file( data_dict, setname ):
    
    for k, v in tqdm( data_dict.items() ):
        
        temp_dict = dict()
        temp_dict[k] = v
        write_parition( temp_dict, setname, k )

In [36]:
# Get LF combinations based on partitions
# train_o_candidates, test_o_ebm_candidates, test_o_ebm_corr_candidates

def get_part_candidates(partitioned_d_umls, cands):

    part_cand_dict = dict()

    for i, partition in enumerate(  tqdm(partitioned_d_umls) ):
        
        #if i <= 3:
        combined_lf = getLFs_modified(partition, cands[0], len( data_df.tokens ))
        assert len( combined_lf ) == (i+1)

        part_cand_dict[i] = combined_lf

    return part_cand_dict

Getting candidate partitions for training set


100%|██████████| 127/127 [8:38:45<00:00, 245.08s/it]  


In [ ]:
print( 'Getting candidate partitions for training set' )
#part_cand_train = get_part_candidates(partitioned_o_umls_fuzzy, train_o_candidates)

print( 'Getting candidate partitions for test ebm set' )
#part_cand_test_ebm = get_part_candidates(partitioned_o_umls_fuzzy, test_o_ebm_candidates)

print( 'Getting candidate partitions for test ebm set corrected' )
#part_cand_test_ebm_corr = get_part_candidates(partitioned_o_umls_fuzzy, test_o_ebm_corr_candidates)

print( 'Getting candidate partitions for test physio set' )
#part_cand_test_physio_corr = get_part_candidates(partitioned_o_umls_fuzzy, test_o_physio_candidates)

In [37]:
#part2file( part_cand_train, 'training_ebm_candidate_generation' )

100%|██████████| 127/127 [1:47:21<00:00, 50.72s/it]


In [42]:
#part2file( part_cand_test_ebm, 'test_ebm_candidate_generation' )

100%|██████████| 127/127 [04:26<00:00,  2.10s/it]


In [43]:
#part2file( part_cand_test_ebm_corr, 'test_ebm_anjani_candidate_generation' )

100%|██████████| 127/127 [04:26<00:00,  2.10s/it]


In [44]:
#part2file( part_cand_test_physio_corr, 'test_physio_candidate_generation' )

100%|██████████| 127/127 [04:26<00:00,  2.10s/it]


In [38]:
# Read the partitions from the JSON files
from os import listdir
from os.path import isfile, join


training_file = f'/mnt/nas2/results/Results/systematicReview/distant_pico/partitions/{candgen_version}/{picos}/training_ebm_candidate_generation/'
test_ebm_file = f'/mnt/nas2/results/Results/systematicReview/distant_pico/partitions/{candgen_version}/{picos}/test_ebm_candidate_generation/'
test_ebm_corr_file = f'/mnt/nas2/results/Results/systematicReview/distant_pico/partitions/{candgen_version}/{picos}/test_ebm_anjani_candidate_generation/'
test_physio_file = f'/mnt/nas2/results/Results/systematicReview/distant_pico/partitions/{candgen_version}/{picos}/test_physio_candidate_generation/'

In [39]:
def mv(L, break_ties, abstain=-1):
    """Simple majority vote"""
    from statistics import mode
    y_hat = []
    for row in L:
        # get non abstain votes
        row = row[row != abstain]
        try:
            l = mode(row)
        except:
            l = break_ties
        y_hat.append(l)
    return np.array(y_hat).astype(np.int)

In [40]:
def dict2metrics(cr_dict):
    
    # accuracy: 89.70 | precision: 14.80 | recall: 36.10 | f1: 20.99 | f1_macro: 57.74
    
    accuracy = round( cr_dict['accuracy'] * 100, 2) 
    precision = round( cr_dict['1']['precision'] * 100, 2) 
    recall = round( cr_dict['1']['recall'] * 100, 2) 
    f1 = round( cr_dict['1']['f1-score'] * 100, 2) 
    f1_macro = round( cr_dict['macro avg']['f1-score'] * 100, 2) 
    
    string2return = 'accuracy: ' + str(accuracy) + ' | ' + 'precision: ' + str(precision) + ' | ' + 'recall: ' +  str(recall) + ' | ' + 'f1: ' + str(f1) + ' | ' + 'f1_macro: ' + str(f1_macro)
    
    return string2return

In [41]:
def grid_search(model_class,
                model_class_init,
                param_grid,
                train=None,
                dev=None,
                other_train=None,
                n_model_search=5,
                val_metric='f1_macro',
                seed=0,
                checkpoint_gt_mv=True,
                tag_fmt_ckpnt='IO'):
    
    
    """Simple grid search helper function
    Parameters
    ----------
    model_class
    model_class_init
    param_grid
    train
    dev
    n_model_search
    val_metric
    seed

    Returns
    -------
    """
    
    L_train, Y_train = train
    L_dev, Y_dev = dev

    # sample configs
    params = sample_param_grid(param_grid, seed)[:n_model_search]

    defaults = {'seed': seed}
    best_score, best_score_mv, best_config = 0.0, 0.0, None
    print(f"Grid search over {len(params)} configs")

    for i, config in enumerate(params):
        print(f'[{i}] Label Model')
        config = dict(zip(param_grid.keys(), config))
        config.update({param: value for param, value in defaults.items() if param not in config})

        model = model_class(**model_class_init)
        model.fit(L_train, Y_dev, **config)
        
        y_pred = model.predict(L_dev)
        
        if tag_fmt_ckpnt == 'IO':
            y_gold = np.array([0 if y == 0 else 1 for y in Y_dev])
        else:
            y_gold = Y_dev
            
            
        if -1 in y_pred:
            print("Label model predicted -1 (TODO: this happens inconsistently)")
            continue
            
        # use internal label model scorer to score the prediction
        metrics = model.score(L=L_dev,
                              Y=y_gold,
                              metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                              tie_break_policy='random')

        # compare learned model against MV on same labeled dev set
        # skip if LM less than MV
        #if checkpoint_gt_mv:
        mv_y_pred = mv(L_dev, 0)
        cr_mv = classification_report( y_gold, mv_y_pred, digits=4, output_dict = True  )
        mv_metrics = cr_mv['macro avg']['f1-score']

        #if metrics[val_metric] < mv_metrics:
        #    continue
                
        if not best_score or metrics[val_metric] > best_score[val_metric]:
            print(config)
            best_score = metrics
            best_score_mv = mv_metrics
            best_config = config
            
            # print training set score if we have labeled data
            if np.any(Y_train):
                y_pred = model.predict(L_train)

                if tag_fmt_ckpnt == 'IO':
                    y_gold = np.array([0 if y == 0 else 1 for y in Y_train])
                else:
                    y_gold = Y_train

                metrics = model.score(L=L_train,
                                      Y=y_gold,
                                      metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                      tie_break_policy='random')

                print('[TRAIN] {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in metrics.items()])))

            print('[DEV]   {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in best_score.items()])))
            print( '[DEVMV]   {}'.format( dict2metrics(cr_mv) ) )
            print('-' * 88)
            
            
    # retrain best model
    print('BEST')
    print(best_config)
    model = model_class(**model_class_init)   
    model.fit(L_train, Y_dev, **best_config)
    
    return model, best_config, best_score, best_score_mv

In [ ]:
def predict(part, cands, best_model, gt_labels, exp_l):
    
    if exp_l == 'UMLS':
    
        combined_lf = getLFs( part, cands[0], len(gt_labels) ) # level 1 UMLS
        assert len(part) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(part) )
        
    elif exp_l == 'UMLS_Ontology':
    
        combined_lf = getLFs( part, cands[0], len(gt_labels) ) # level 1 UMLS
        assert len(part) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(part) )
        combined_lf.extend( list(cands[1].values()) ) # level 2 non-UMLS
        
    elif exp_l == 'UMLS_Ontology_Rules':

        combined_lf = getLFs( part, cands[0], len(gt_labels) ) # level 1 UMLS
        assert len(part) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(part) )
        combined_lf.extend( list(cands[1].values()) ) # level 2 non-UMLS
        combined_lf.extend( list(cands[2].values()) ) # level 3 DS - Heur
        combined_lf.extend( list(cands[3].values()) ) # level 4 dict - Heur 
        combined_lf.extend( list(cands[4].values()) ) # level 4 ReGeX, Abb - Heur


    L = np.array( combined_lf )
    L = np.transpose(L)
    
    predictions_probablities = best_model.predict_proba(L)
    predictions = best_model.predict(L)
    groundtruth = np.array(gt_labels) 
    groundtruth_ = [1 if x != 0 else x for x in gt_labels] # XXX if "test_ebm_correct"
    groundtruth = np.array(groundtruth_)

    cr = classification_report( groundtruth, predictions, digits=4 )
    print( cr )
    
    return predictions_probablities

In [57]:
def train( candidates, Y_d, picos, paramgrid, mode ):
    
    train_cands, test_cands, test_corr_cands, test_physio = candidates
  
    gold_labels = ''
    gold_labels_fine = ''
    
    
    model_class_init = {
        'cardinality': 2, 
        'verbose': True
    }

    num_hyperparams = functools.reduce(lambda x,y:x*y, [len(x) for x in param_grid.values()])
    print("Hyperparamater Search Space:", num_hyperparams)
    n_model_search = 50
    

    '''#########################################################################
    # Choosing the number of LF's from UMLS all
    #########################################################################'''
    
    partition_contributions = dict()
    partition_contributions_mv = dict()
    
    for l in exp_level:
        print( 'Executing the experiment level: ', l )
        
        best_f1_macro = 0.0
        best_overall_model = ''
        best_overall_config = ''
        best_overall_partition = 0
        overall_L = ''
        
        best_score_parts = [ ]
        best_score_mv_parts = [ ]
        
        # list all files from a dir
        onlyfiles = sorted( [ int(str(f).replace('.json', '')) for f in listdir(training_file) if isfile(join(training_file, f))] )
        
        for partition in onlyfiles:
            
            partition_file = training_file + str(partition) + str('.json')
            
            with open( partition_file, 'r' ) as rf:        
            
                partition_i = json.load(rf)           
                                
                for k, v in partition_i.items():

                    if l == 'UMLS' and str(k) < 2:
                        continue

                    if l == 'UMLS':

                        combined_lf = list(v)
                        assert int(k)+1 == len(combined_lf)
                        print( 'Total number of UMLS partitions: ', len(combined_lf) )

                    if l == 'UMLS_Ontology':

                        combined_lf = list(v)
                        assert int(k)+1 == len(combined_lf)
                        print( 'Total number of UMLS partitions: ', len(combined_lf) )
                        combined_lf.extend( list(train_cands[1].values()) ) # level 2 non-UMLS

                    if l == 'UMLS_Ontology_Rules':

                        combined_lf = list(v)
                        assert int(k)+1 == len(combined_lf)
                        print( 'Total number of UMLS partitions: ', len(combined_lf) )
                        combined_lf.extend( list(train_cands[1].values()) ) # level 2 non-UMLS
                        combined_lf.extend( list(train_cands[2].values()) ) # level 3 DS - Heur
                        combined_lf.extend( list(train_cands[3].values()) ) # level 4 dict - Heur 
                        combined_lf.extend( list(train_cands[4].values()) ) # level 4 ReGeX, Abb - Heur


                    L = np.array( combined_lf )
                    L = np.transpose(L)
                    L_train, L_val = train_test_split(L, test_size=0.20, shuffle=False)
                    Y_train, Y_val = train_test_split( np.array(Y_d[0]), test_size=0.20, shuffle=False)
                    Y_train_fine, Y_val_fine = train_test_split( np.array(Y_d[1]), test_size=0.20, shuffle=False)

                    # convert the fine labels to 0 and 1
                    Y_train_fine = [1 if x != 0 else x for x in Y_train_fine]
                    Y_val_fine = [1 if x != 0 else x for x in Y_val_fine]

                    Y = np.concatenate([Y_train, Y_val])
                    Y_fine = np.concatenate([Y_train_fine, Y_val_fine])

                    best_model, best_config, best_score, best_score_mv = grid_search(LMsnorkel, 
                                                           model_class_init, 
                                                           paramgrid,
                                                           train = (L_train, Y_train_fine),
                                                           dev = (L_val, Y_val_fine),
                                                           n_model_search=n_model_search, 
                                                           val_metric='f1_macro', 
                                                           seed=seed_i,
                                                           tag_fmt_ckpnt='IO')

                    best_score_parts.append( best_score['f1_macro'] )
                    best_score_mv_parts.append( best_score_mv )

                    if best_score['f1_macro'] > best_f1_macro:
                        best_f1_macro = best_score['f1_macro']
                        best_overall_model = best_model
                        best_overall_config = best_config
                        best_overall_partition = k
                        overall_L = L
                        gold_labels = Y_d[0]
                        gold_labels_fine = Y_d[1]


                    print('Best overall macro F1 score: ', best_f1_macro)
                    print('Best overall configuration: ', best_overall_config)

        partition_contributions[ l ] = best_score_parts
        partition_contributions_mv[ l ] = best_score_mv_parts

        print('Save the best overall model, configuration and partition for this experiment level')
        # Save your model or results
        save_dir = f'/mnt/nas2/results/Results/systematicReview/distant_pico/models/LabelModels/{picos}/v4/{l}/{seed_i}'
        filename = 'stpartition_' + str(best_overall_partition) + '_epoch_' + str(best_config['n_epochs'])
        joblib.dump(best_overall_model, f'{save_dir}/{filename}.pkl') 
        joblib.dump(best_overall_config, f'{save_dir}/{filename}.json')
        
    return partition_contributions_mv, partition_contributions

In [47]:
# Seed 0
partition_contributions_mv_0, partition_contributions_0 = train( candidates = (train_o_candidates, test_o_ebm_corr_candidates, test_o_ebm_candidates, test_o_physio_candidates), Y_d = Y_o, picos = 'o', paramgrid = param_grid, mode = 'pred')

Hyperparamater Search Space: 336
Executing the experiment level:  UMLS_Ontology_Rules
Total number of UMLS partitions:  1
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 76.68 | precision: 19.60 | recall: 49.72 | f1: 28.12 | f1_macro: 57.10
[DEV]   accuracy: 76.72 | precision: 20.04 | recall: 50.39 | f1: 28.68 | f1_macro: 57.39
[DEVMV]   accuracy: 66.84 | precision: 16.79 | recall: 65.02 | f1: 26.69 | f1_macro: 52.63
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 78.47 | precision: 20.27 | recall: 45.92 | f1: 28.13 | f1_macro: 57.73
[DEV]   accuracy: 78.52 | precision: 20.76 | recall: 46.60 

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.7, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 76.65 | precision: 19.15 | recall: 47.95 | f1: 27.37 | f1_macro: 56.73
[DEV]   accuracy: 76.62 | precision: 19.57 | recall: 48.80 | f1: 27.94 | f1_macro: 56.99
[DEVMV]   accuracy: 71.87 | precision: 17.75 | recall: 55.85 | f1: 26.94 | f1_macro: 54.76
----------------------------------------------------------------------------------------
[7] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.61 | precision: 21.83 | recall: 30.49 | f1: 25.44 | f1_macro: 58.12
[DEV]   accuracy: 83.47 | precision: 21.99 | recall: 30.62 | f1: 25.60 | f1_macro: 58.15
[DEVMV]   accuracy: 71.87 | precision: 17.75 | recall: 55.85 | f1: 26.94 | f1_macro: 54.76
----------------------------------------------------------------------------------------
[8] Label Model
[9] 

[2] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 73.16 | precision: 17.60 | recall: 52.27 | f1: 26.33 | f1_macro: 54.96
[DEV]   accuracy: 73.18 | precision: 17.79 | recall: 52.15 | f1: 26.53 | f1_macro: 55.06
[DEVMV]   accuracy: 72.52 | precision: 17.8 | recall: 54.17 | f1: 26.8 | f1_macro: 54.94
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 76.28 | precision: 19.07 | recall: 48.90 | f1: 27.44 | f1_macro: 56.63
[DEV]   accuracy: 76.25 | precision: 19.47 | recall: 49.67 | f1: 27.98 | f1_macro: 56.88
[DEVMV]   accuracy: 72.52 | precision: 17.8 | recall: 54.17 | f1: 26.8 | f1_macro: 54.94
------------------------------------------------------------------

[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Best overall macro F1 score:  0.5992289338005614
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Total number of UMLS partitions:  7
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_i

[16] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 84.25 | precision: 22.78 | recall: 30.00 | f1: 25.90 | f1_macro: 58.54
[DEV]   accuracy: 84.14 | precision: 22.93 | recall: 29.97 | f1: 25.98 | f1_macro: 58.55
[DEVMV]   accuracy: 73.21 | precision: 17.83 | recall: 52.24 | f1: 26.59 | f1_macro: 55.1
----------------------------------------------------------------------------------------
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label M

...

[7] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 82.11 | precision: 20.30 | recall: 32.50 | f1: 24.99 | f1_macro: 57.42
[DEV]   accuracy: 82.05 | precision: 20.63 | recall: 32.76 | f1: 25.32 | f1_macro: 57.56
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 81.95 | precision: 20.49 | recall: 33.59 | f1: 25.46 | f1_macro: 57.59
[DEV]   accuracy: 81.93 | precision: 2

[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Best overall macro F1 score:  0.5992289338005614
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Total number of UMLS partitions:  120
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.11 | precision: 19.97 | recall: 27.94 | f1: 23.29 | f1_macro: 56.90
[DEV]   accuracy: 83.06 | precision: 20.21 | recall: 27.94 | f1: 23.46 | f1_macro: 56.97
[DEVMV]   accuracy: 76.4 | precisio

[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 81.95 | precision: 20.49 | recall: 33.59 | f1: 25.46 | f1_macro: 57.59
[DEV]   accuracy: 81.93 | precision: 20.93 | recall: 34.06 | f1: 25.93 | f1_macro: 57.82
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 81.88 | precision: 20.59 | recall: 34.14 | f1: 25.69 | f1_macro: 57.68
[DEV]   accuracy: 81.84 | precision: 

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.11 | precision: 19.96 | recall: 27.94 | f1: 23.28 | f1_macro: 56.90
[DEV]   accuracy: 83.05 | precision: 20.20 | recall: 27.94 | f1: 23.45 | f1_macro: 56.96
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
[6] Label Model
[7] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 82.11 | precision: 20.30 | recall: 32.50 | f1: 24.99 | f1_macro: 57.42
[DEV]   accuracy: 82.05 | precision: 20.63 | recall: 32.76 | f1: 25.32 | f1_macro: 57.56
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
-----------------

[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 81.88 | precision: 20.59 | recall: 34.14 | f1: 25.69 | f1_macro: 57.68
[DEV]   accuracy: 81.84 | precision: 20.99 | recall: 34.58 | f1: 26.12 | f1_macro: 57.89
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Best overall macro F1 score:  0.599228

In [48]:
# Seed 1

seed_i = 1

partition_contributions_mv_1, partition_contributions_1 = train( candidates = (train_o_candidates, test_o_ebm_corr_candidates, test_o_ebm_candidates, test_o_physio_candidates), Y_d = Y_o, picos = 'o', paramgrid = param_grid, mode = 'pred')

Hyperparamater Search Space: 336
Executing the experiment level:  UMLS_Ontology_Rules
Total number of UMLS partitions:  1
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 86.21 | precision: 22.64 | recall: 20.79 | f1: 21.68 | f1_macro: 57.06
[DEV]   accuracy: 86.14 | precision: 22.78 | recall: 20.59 | f1: 21.63 | f1_macro: 57.01
[DEVMV]   accuracy: 66.84 | precision: 16.79 | recall: 65.02 | f1: 26.69 | f1_macro: 52.63
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 78.73 | precision: 20.34 | recall: 45.21 | f1: 28.06 | f1_macro: 57.79
[DEV]   accuracy: 78.77 | precision: 20.83 | recall: 45.93 | 

[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 87.38 | precision: 27.87 | recall: 23.68 | f1: 25.60 | f1_macro: 59.35
[DEV]   accuracy: 87.30 | precision: 28.13 | recall: 23.63 | f1: 25.68 | f1_macro: 59.37
[DEVMV]   accuracy: 71.87 | precision: 17.75 | recall: 55.85 | f1: 26.94 | f1_macro: 54.76
----------------------------------------------------------------------------------------
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
Best overall macro F1 score:  0.5949074627874122
Best overall conf

[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
Best overall macro F1 score:  0.5979065992675199
Best overall configuration:  {'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
Total number of UMLS partitions:  7
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 83.29 | precision: 21.53 | recall: 31.04 | f1: 25.42 | f1_macro: 58.01
[DEV]   accuracy: 83.23 | precision: 21.78 | recall: 31.08 | f1: 25.61 | f1_macro: 58.08
[DEVMV]   accuracy: 72.93 | precision: 17.75 | recall: 52.67 | f1: 26.55 | f1_macro: 54.98
----------------------------------------------------------------

Total number of UMLS partitions:  9
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 82.86 | precision: 21.00 | recall: 31.44 | f1: 25.18 | f1_macro: 57.75
[DEV]   accuracy: 82.76 | precision: 21.03 | recall: 31.06 | f1: 25.08 | f1_macro: 57.67
[DEVMV]   accuracy: 74.04 | precision: 18.27 | recall: 51.66 | f1: 26.99 | f1_macro: 55.6
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 83.91 | precision: 21.93 | recall: 29.47 | f1: 25.15 | f1_macro: 58.07
[DEV]   accuracy: 83.82 | precision: 22.04 | recall: 29.27 | f1: 25.15 | f1_macro: 58.04
[DEVMV]   accuracy: 74.04 | precision: 18.27 | recall: 51.66 | f1: 26.99 | f1_macro: 55.6
------------------------------------

[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 81.65 | precision: 21.10 | recall: 36.48 | f1: 26.73 | f1_macro: 58.12
[DEV]   accuracy: 81.58 | precision: 21.32 | recall: 36.53 | f1: 26.92 | f1_macro: 58.19
[DEVMV]   accuracy: 74.13 | precision: 18.32 | recall: 51.65 | f1: 27.05 | f1_macro: 55.66
----------------------------------------------------------------------------------------
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
Label model predicted -1 (TODO: this happens inconsistently)
[27] Label Model
[28] Label Model
[29] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'sgd', 'lr_s

...

Total number of UMLS partitions:  118
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 84.62 | precision: 20.72 | recall: 23.93 | f1: 22.21 | f1_macro: 56.84
[DEV]   accuracy: 84.56 | precision: 20.91 | recall: 23.82 | f1: 22.27 | f1_macro: 56.85
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 83.05 | precision: 20.20 | recall: 28.73 | f1: 23.72 | f1_macro: 57.09
[DEV]   accuracy: 82.97 | precision: 20.24 | recall: 28.37 | f1: 23.63 | f1_macro: 57.02
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
------------------------------------

[3] Label Model
Label model predicted -1 (TODO: this happens inconsistently)
[4] Label Model
[5] Label Model
[6] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 81.51 | precision: 20.61 | recall: 35.60 | f1: 26.11 | f1_macro: 57.77
[DEV]   accuracy: 81.44 | precision: 20.96 | recall: 36.01 | f1: 26.49 | f1_macro: 57.94
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 81.51 | precision: 20.61 | recall: 35.61 | f1: 26.11 | f1_m

[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 81.51 | precision: 20.61 | recall: 35.61 | f1: 26.11 | f1_macro: 57.77
[DEV]   accuracy: 81.44 | precision: 20.96 | recall: 36.02 | f1: 26.50 | f1_macro: 57.94
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[19] Label Model
[20] Label Model
[21] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 81.77 | precision: 20.68 | recall: 34.81 | f1: 25.94 | f1_macro: 57.78
[DEV]   accuracy: 81.72 | precision: 21.07 | recall: 35.24 | f1: 26.37 | f1_macro: 57.97
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46

[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
Label model predicted -1 (TODO: this happens inconsistently)
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
Best overall macro F1 score:  0.5979065992675199
Best overall configuration:  {'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
Total number of UMLS partitions:  125
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.8, 'optimi

Total number of UMLS partitions:  127
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 2000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 84.61 | precision: 20.71 | recall: 23.94 | f1: 22.21 | f1_macro: 56.83
[DEV]   accuracy: 84.54 | precision: 20.88 | recall: 23.82 | f1: 22.26 | f1_macro: 56.84
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1}
[TRAIN] accuracy: 83.05 | precision: 20.20 | recall: 28.73 | f1: 23.72 | f1_macro: 57.09
[DEV]   accuracy: 82.97 | precision: 20.25 | recall: 28.37 | f1: 23.63 | f1_macro: 57.02
[DEVMV]   accuracy: 76.4 | precision: 18.94 | recall: 46.99 | f1: 27.0 | f1_macro: 56.46
------------------------------------

In [ ]:
# Seed 42

seed_i = 42

partition_contributions_mv_42, partition_contributions_42 = train( candidates = (train_o_candidates, test_o_ebm_corr_candidates, test_o_ebm_candidates, test_o_physio_candidates), Y_d = Y_o, picos = 'o', paramgrid = param_grid, mode = 'pred')

Hyperparamater Search Space: 336
Executing the experiment level:  UMLS_Ontology_Rules
Total number of UMLS partitions:  1
Grid search over 50 configs
[0] Label Model


In [ ]:
for x, y in partition_contributions_mv_1.items():
    for i in y:
        print(i)